# 🛳️ Projeto Titanic – Kaggle

Este notebook tem como objetivo analisar os dados do Titanic, tratar os valores ausentes e treinar modelos para prever quais passageiros sobreviveram.


## 📁 1. Importação de bibliotecas e leitura dos dados

Vamos importar o pandas e carregar o arquivo `train.csv` para começar a análise.

In [1]:
# importar biblioteca pandas
import pandas as pd

# carregar o arquivo
df = pd.read_csv('train.csv')

# visualizar as 5 primeiras linhas
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 🔍 2. Análise inicial dos dados

Vamos ver como os dados estão organizados, identificar colunas importantes e verificar valores ausentes.

In [2]:
# ver informações gerais das colunas
df.info()

# ver total de valores nulos por colunas
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

## 🧩 3. Criar coluna NoAgeNoCabin

Antes de preencher a idade, vamos criar uma coluna chamada `NoAgeNoCabin` que marca os passageiros que não têm `Age` **nem** `Cabin`. A ausência dessas duas informações parece estar muito associada à não sobrevivência.

In [3]:
# criar coluna 'NoAgeNoCabin' (true para quem não tem Age e nem Cabin informados)
df ['NoAgeNoCabin'] = df['Age'].isnull() & df['Cabin'].isnull()

# ver as primeiras linhas para conferir
df[['Age', 'Cabin', 'NoAgeNoCabin']].head()


,Age,Cabin,NoAgeNoCabin
0,22.0,NaN,False
1,38.0,C85,False
2,26.0,NaN,False
3,35.0,C123,False
4,35.0,NaN,False


In [4]:
# ver quantas pessoas se encaixam nesse caso
df['NoAgeNoCabin'].sum()

158

In [5]:
# filtrar apenas quem não tem age nem cabin
sem_age_cabin = df[df['NoAgeNoCabin'] == True]

# contar quantos sobreviveram (1) e quantos não (0)
sem_age_cabin['Survived'].value_counts()

0    117
1     41
Name: Survived, dtype: int64

## 📊 4. Análise dos passageiros com Age ausente

Vamos analisar quantos passageiros com idade ausente estão em cada classe (`Pclass`) e seu sexo (`Sex`). Isso vai nos ajudar a escolher valores apropriados para preencher.

In [6]:
# filtrar apenas passageiros sem idade
sem_idade = df[df["Age"].isnull()]

# ver contagem por classe
print("Classe dos passageiros sem idade:")
print(sem_idade["Pclass"].value_counts())

# ver contagem por sexo
print("\nSexo dos passageiros sem idade:")
print(sem_idade["Sex"].value_counts())

# cruzar sexo e classe
print("\nSexo e classe dos passageiros sem idade:")
print(sem_idade.groupby(["Sex", "Pclass"]).size())

Classe dos passageiros sem idade:
3    136
1     30
2     11
Name: Pclass, dtype: int64

Sexo dos passageiros sem idade:
male      124
female     53
Name: Sex, dtype: int64

Sexo e classe dos passageiros sem idade:
Sex     Pclass
female  1          9
        2          2
        3         42
male    1         21
        2          9
        3         94
dtype: int64


## 🧮 5. Calcular e usar medianas para preencher Age

Vamos calcular a mediana da idade dos passageiros da 3ª classe, separando por sexo. Em seguida, vamos usar essas medianas para preencher os valores ausentes de `Age`.

In [7]:
# calcular as medianas da idade para cada sexo na 3 classe
mediana_homens_classe3 = df[(df["Sex"] == "male") & (df["Pclass"] == 3)]["Age"].median()
mediana_mulheres_classe3 = df[(df["Sex"] == "female") & (df["Pclass"] == 3)]["Age"].median()

print(f"Mediana homens classe 3: {mediana_homens_classe3}")
print(f"Mediana mulheres classe 3: {mediana_mulheres_classe3}")

Mediana homens classe 3: 25.0
Mediana mulheres classe 3: 21.5


## ✏️ 6. Preenchendo valores ausentes de Age com valores fixos

Para avançar no projeto, vamos usar uma abordagem simples: preencher as idades ausentes com valores fixos baseados no sexo do passageiro:

- Homens → 25
- Mulheres → 21

In [8]:
# preencher homens com idade 25
df.loc[(df["Age"].isnull()) & (df["Sex"] == "male"), "Age"] = 25

# preencher mulheres com idade 21
df.loc[(df["Age"].isnull()) & (df["Sex"] == "female"), "Age"] = 21

In [9]:
# confirmar se não restam valores ausentes em Age
df['Age'].isnull().sum()

0

## 🧹 7. Revisar colunas disponíveis e preparar para o modelo

Vamos verificar todas as colunas do nosso DataFrame e decidir quais serão usadas como **entradas do modelo (features)** e qual será o **alvo (target)**. Também vamos transformar colunas de texto em números quando necessário.

In [10]:
# ver todas as colunas e tipos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   891 non-null    int64  
 1   Survived      891 non-null    int64  
 2   Pclass        891 non-null    int64  
 3   Name          891 non-null    object 
 4   Sex           891 non-null    object 
 5   Age           891 non-null    float64
 6   SibSp         891 non-null    int64  
 7   Parch         891 non-null    int64  
 8   Ticket        891 non-null    object 
 9   Fare          891 non-null    float64
 10  Cabin         204 non-null    object 
 11  Embarked      889 non-null    object 
 12  NoAgeNoCabin  891 non-null    bool   
dtypes: bool(1), float64(2), int64(5), object(5)
memory usage: 84.5+ KB


## 🧹 8. Remover colunas desnecessárias

Vamos excluir colunas que não serão utilizadas no modelo:
- `Name`, `Ticket`, `Cabin`, `PassengerId` → não são úteis para prever sobrevivência
- `Embarked` → será excluída para evitar tratar nulos neste primeiro projeto

In [11]:
# excluir colunas desnecessárias
df = df.drop(columns=['Name', 'Ticket', 'Cabin', 'PassengerId', 'Embarked'])

## 🔁 9. Transformar valores de texto em números

O modelo de machine learning só entende números. Vamos transformar a coluna `Sex` em valores binários:
- `male` → 1
- `female` → 0

In [12]:
# transformar male em 1 e female em 0
df['Sex'] = df['Sex'].map({'male': 1, 'female': 0})

# transformar True em 1 e False em 0
df['NoAgeNoCabin'] = df['NoAgeNoCabin'].astype(int)

In [13]:
# visualizar as 5 primeiras linhas para conferir se funcionou
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,NoAgeNoCabin
0,0,3,1,22.0,1,0,7.2500,0
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,0
3,1,1,0,35.0,1,0,53.1000,0
4,0,3,1,35.0,0,0,8.0500,0


## 🎯 10. Separar dados em X (features) e y (alvo)

Agora vamos separar os dados em:
- `X`: todas as colunas usadas para treinar o modelo
- `y`: a coluna `Survived`, que é o que queremos prever

In [14]:
# separando dados de entrada e de saida (atribuindo todas as colunas, exceto 'Survived' à variável x, e atribuindo apenas essa coluna à variável y')
X = df.drop(columns=['Survived'])
y = df['Survived']

## 🧪 11. Separar os dados em treino e teste

Vamos dividir os dados em dois conjuntos:
- `X_train`, `y_train`: usados para treinar o modelo
- `X_test`, `y_test`: usados para avaliar a performance do modelo

In [15]:
# importar função train_test_split de um módulo da biblioteca sklearn
from sklearn.model_selection import train_test_split

# dividir os dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 🌳 12. Treinar o modelo com árvore de decisão

Vamos treinar um modelo de árvore de decisão usando os dados de treino.

In [16]:
# importar função de um módulo da biblioteca sklearn
from sklearn.tree import DecisionTreeClassifier

# criar o modelo
modelo = DecisionTreeClassifier()

# treinar com os dados de treino
modelo.fit(X_train, y_train)

DecisionTreeClassifier()

## 📈 13. Avaliar o modelo

Vamos calcular a **acurácia** do modelo usando os dados de teste.

In [17]:
# importar função da sklearn
from sklearn.metrics import accuracy_score

# fazer previsões
y_pred = modelo.predict(X_test)

# avaliar acurácia
acuracia = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo: {acuracia:.2%}")

Acurácia do modelo: 77.65%


## 🔃14. Criar uma função de pré-processamento
Vamos criar uma função com os comandos já utilizados anteriormente.

In [18]:
# criar a função
def preprocess (df):

# criar coluna 'NoAgeNoCabin' (true para quem não tem Age e nem Cabin informados)
 df ['NoAgeNoCabin'] = df['Age'].isnull() & df['Cabin'].isnull()

# preencher idade com base no sexo, com a mediana das idades da 3 classe
 df.loc[(df['Age'].isnull()) & (df['Sex'] == 'male'), 'Age'] = 25
 df.loc[(df['Age'].isnull()) & (df['Sex'] == 'female'), 'Age'] = 21

# excluir colunas desnecessárias
 df = df.drop(columns=['Name', 'Ticket', 'Cabin', 'PassengerId', 'Embarked'])

# transformar male em 1 e female em 0
 df['Sex'] = df['Sex'].map({'male': 1, 'female': 0})

#transformar True em 1 e False em 0
 df['NoAgeNoCabin'] = df['NoAgeNoCabin'].astype(int)

 return df
